# Root Finders and Minimizers in Python
### by [Richard W. Evans](https://sites.google.com/site/rickecon/), May 2017

The code in this Jupyter notebook was written using Python 3.5.

## 1. General characterization of maximization/minimization problem
Root finders and minimizers are the two key tools for solving numerical optimization problems. We will define the general formulation of an optimization problem as a minimization problem in the following way,

$$ \min_{x}\: f\left(x,z|\theta\right) \quad\text{s.t}\quad g(x,z|\theta)\geq 0 \quad\text{and}\quad h(x) = 0 $$

where $f$ is a system of potentially nonlinear equations that are a function of the vectors of (potentially dynamic)  variables $x$ and $z$ and parameter vector $\theta$, subject to the vector of inequality constraints $g$ and the vector of equality constraints $h$.

A computational algorithm that searches for the value of $x$ that minimizes the problem above is called a **minimizer**. Sometimes the solution to the minimization problem above can be written as a system of equations in $x$.

$$ \hat{x} = x: \quad \phi(x|z,\theta) = 0 $$

The maximization problem can be reduced to this system of characterizing equations when the inequality constraints can be shown to never bind (interior solution). A computational algorithm that searches for the value $\hat{x}$ that sets the value of each equation in the system $\phi(x|z)$ to 0 is called a **root finder**. 

## 2. Root Finders
Suppose the solution to a system of $N$ nonlinear equations $\phi(x|z,\theta)=0$ is the $N\times 1$ vector $\hat{x}$. 


### 2.1. Univariate root finders
Suppose that $x$ is a scalar such that the system of equations is one equation $\phi(x|\theta)=0$ and one unknown $x$. Let's assume the simple cubic polynomial functional form.

$$\phi(x|\theta)= x^3 + x + \theta $$

The roots of this function are all the values of $x$ such that $\phi(x|\theta)=0$, or rather $x^3 + x + \theta$. We first define a Python function `cubic_pol()` that takes as inputs a value for $x$ and a value for $\theta$ and returns the value of the function.

In [ ]:
def phi_pol(xvals, theta):
    '''
    --------------------------------------------------------------------
    This function returns the value phi(x,theta) given x and theta,
    where phi(x,theta) = (x ** 3) + x + theta
    --------------------------------------------------------------------
    INPUTS:
    xvals = scalar or (N,) vector, value or values for x
    args  = length 1 tuple, (theta)
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION: None
    
    OBJECTS CREATED WITHIN FUNCTION:
    theta = scalar, constant in the phi function
    phi   = scalar or (N,) vector, value of phi(xvals, theta)
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: phi
    --------------------------------------------------------------------
    '''
    phi = (xvals ** 3) + xvals + theta
    
    return phi

We can explore this function $\phi(x|\theta)$ graphically by plotting it for a given value of $\theta$ and for many values of $x$. The following plot is for $N=100$ values of $x$ between -10 and 10 and for $\theta=10$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
# This next command is specifically for Jupyter Notebook
%matplotlib notebook

theta = 10
xmin = -10
xmax = 10
N = 100
xvals = np.linspace(xmin, xmax, N)
phi_vals = phi_pol(xvals, theta)

# Plot the resulting phi values
fig, ax = plt.subplots()
plt.plot(xvals, phi_vals)
minorLocator = MultipleLocator(1)
ax.xaxis.set_minor_locator(minorLocator)
plt.grid(b=True, which='major', color='0.65', linestyle='-')
plt.title('Phi function for theta=10 and x=[-10,10]', fontsize=20)
plt.xlabel(r'$x$')
plt.ylabel(r'$\phi(x|\theta=10)$')

From the plot above, it looks like the function $\phi(x|\theta=10)$ has one root somewhere in the neighborhood of $x\approx -2$. Plugging `x=2` into the function reveals that it is identically the root. So we know what the answer should be. Let's see how close the root finding algorithms get.

Python's `Scipy` library has a sub-library called `scipy.optimize`. The `scipy.optimize` sub-library has all of Python's most standard univariate and multivariate root finder algorithms as well as Python's most standard minimizer algorithms. Scrolling a little more than halfway down the main [`scipy.optimize` reference page](https://docs.scipy.org/doc/scipy/reference/optimize.html), we find the links to the documentation for the 5 univariate root finder algorithms in `scipy.optimize`. These are:

* [`brentq()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html#scipy.optimize.brentq)
* [`brenth()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brenth.html#scipy.optimize.brenth)
* [`ridder()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.ridder.html#scipy.optimize.ridder)
* [`bisect()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html#scipy.optimize.bisect)
* [`newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html#scipy.optimize.newton)

The first four univariate root finders require a known bracketing interval for the root. That is, the user must know that $\hat{x}\in[a, b]$. For the `newton()` method, the bracketing interval is not required. We will solve for this root with the `bisect()` method and with the `newton()` method.

In [ ]:
import scipy.optimize as opt

a =-5
b = 0

(xhat, result) = opt.bisect(phi_pol, a, b, args=(theta), full_output=True)

print('xhat: ', xhat)
print(result)

As we saw in the Figure of the $\phi(x|\theta=10)$ function, the root for this function is very close to $\hat{x}\approx 2$.

One caveat with the bisection method is that it will only find one root in the bracketing interval $[a,b]$. If the function has two roots in the interval, the `bisect()` method will only find one of them. This is true of all five univariate root finder methods as well as all minimizers and multivariate root finders in `scipy.optimize`. They will only return one root of minimum. Whether that root is unique or whether the minimum is a global min rather than a local min depends on how well the researcher understands the function.

The `newton()` method uses a standard Newton-Raphson hill climbing (gradient descent) method to find the root of the function. Rather than needing to know the bracketing interval where the root exists, you must supply the function with an initial guess.

In [ ]:
x_init = 10.0

xhat = opt.newton(phi_pol, x_init, args=(theta,))
print(xhat)

One thing to note here is that the precision of the tolerance matters. That is, I can get a slightly different answer if I start the search from `x_init=-10`. In fact, when I start below the actual value I get the analytical solution $\hat{x}=-2.0$. However, the difference between the two answers is extremely small.

### 2.2. Multivariate root finders
For our example of multivariate root finders, let's use the two characterizing equations for optimal lifetime savings from the 3-period-lived agent overlapping generations model with exogenous labor supply. In this case, the solution $x$ will be a $(2\times 1)$ vector of savings for middle age $b_2$ and old age $b_3$ given interest rates ($r_2, r_3$) and wages ($w_1, w_2, w_3$) over the lifetime and utility function parameters ($\beta, \sigma$).

$$ \phi(b_2, b_3|r_2, r_3, w_1, w_2, w_3, \beta, \sigma) = 0 $$

$$ \Rightarrow \qquad\:\:\:\, (w_1 - b_2)^{-\sigma} - \beta(1 + r_2)([1 + r_2]b_2 + w_2 - b_3)^{-\sigma} = 0 $$
$$ \Rightarrow ([1 + r_2]b_2 + w_2 - b_3)^{-\sigma} - \beta(1 + r_3)([1 + r_3]b_3 + w_3)^{-\sigma} = 0 $$

This is two equations and two unknowns ($b_2, b_3$). Everything other than the two savings levels is known in the two equations. Further, Inada conditions bound the solution for consumption strictly away from $c_t\leq 0$, which means that the two equation system should be solvable using an unconstrained root finder for ($b_2, b_3$).

We first write some functions that compute the Euler errors shown above given the values for lifetime savings ($b_2, b_3$), interest rates ($r_2, r_3$), wages ($w_1, w_2, w_3$), and utility function parameters ($\beta, \sigma$).

In [ ]:
def get_ct(bt, btp1, rt, wt):
    '''
    --------------------------------------------------------------------
    This function returns the value of current period consumption given
    b_t, b_{t+1}, r_t, and w_t.
    --------------------------------------------------------------------
    INPUTS:
    bt   = scalar, current period savings
    btp1 = scalar, savings for next period
    rt   = scalar > 0, current period interest rate
    wt   = scalar > 0, current period wage
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION: None
    
    OBJECTS CREATED WITHIN FUNCTION:
    ct = scalar, current period consumption
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: ct
    --------------------------------------------------------------------
    '''
    ct = (1 + rt) * bt + wt - btp1
    
    return ct


def MU_stitch(ct, sigma):
    '''
    --------------------------------------------------------------------
    Generate marginal utility of consumption with CRRA consumption
    utility and stitched function at lower bound such that the new
    hybrid function is defined over all consumption on the real
    line but the function has similar properties to the Inada condition.

    u'(c) = c ** (-sigma) if c >= epsilon
          = g'(c) = 2 * b2 * c + b1 if c < epsilon

        such that g'(epsilon) = u'(epsilon)
        and g''(epsilon) = u''(epsilon)

        u(c) = (c ** (1 - sigma) - 1) / (1 - sigma)
        g(c) = b2 * (c ** 2) + b1 * c + b0
    --------------------------------------------------------------------
    INPUTS:
    ct    = scalar, current consumption
    sigma = scalar >= 1, coefficient of relative risk aversion for CRRA
            utility function: (c**(1-sigma) - 1) / (1 - sigma)

    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION: None

    OBJECTS CREATED WITHIN FUNCTION:
    epsilon  = scalar > 0, positive value close to zero
    c_cnstr  = boolean, =True if ct < epsilon
    b1       = scalar, intercept value in linear marginal utility
    b2       = scalar, slope coefficient in linear marginal utility
    MU_c     = scalar, marginal utility of consumption
    
    FILES CREATED BY THIS FUNCTION: None

    RETURNS: MU_c
    --------------------------------------------------------------------
    '''
    epsilon = 0.0001
    c_cnstr = ct < epsilon
    if c_cnstr:
        b2 = (-sigma * (epsilon ** (-sigma - 1))) / 2
        b1 = (epsilon ** (-sigma)) - 2 * b2 * epsilon
        MU_c = 2 * b2 * ct + b1
    else:
        MU_c = ct ** (-sigma)
    
    return MU_c


def get_EulErrs(bvec, *args):
    '''
    --------------------------------------------------------------------
    INPUTS:
    bvec = (2,) vector, values for lifetime savings (b2, b3)
    args = length 7 tuple, (r2, r3, w1, w2, w3, beta, sigma)
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION:
        get_ct()
        MU_stitch()
    
    OBJECTS CREATED WITHIN FUNCTION:
    r2    = scalar > 0, interest rate in period 2
    r3    = scalar > 0, interest rate in period 3
    w1    = scalar > 0, wage in period 1
    w2    = scalar > 0, wage in period 2
    w3    = scalar > 0, wage in period 3
    beta  = scalar in (0, 1), discount factor
    sigma = scalar >= 1, coefficient of relative risk aversion
    c1    = scalar, consumption in period 1
    c2    = scalar, consumption in period 2
    c3    = scalar, consumption in period 3
    MU_c1 = scalar > 0, marginal utility of consumption in period 1
    MU_c2 = scalar > 0, marginal utility of consumption in period 2
    MU_c3 = scalar > 0, marginal utility of consumption in period 3
    err1  = scalar, Euler error for savings decision b2
    err2  = scalar, Euler error for savings decision b3
    err_vec = (2,) vector, Euler errors from two Euler equations
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: err_vec
    --------------------------------------------------------------------
    '''
    b2, b3 = bvec
    r2, r3, w1, w2, w3, beta, sigma = args
    c1 = get_ct(0.0, b2, 0.0, w1)
    c2 = get_ct(b2, b3, r2, w2)
    c3 = get_ct(b3, 0.0, r3, w3)
    MU_c1 = MU_stitch(c1, sigma)
    MU_c2 = MU_stitch(c2, sigma)
    MU_c3 = MU_stitch(c3, sigma)
    err1 = MU_c1 - beta * (1 + r2) * MU_c2
    err2 = MU_c2 - beta * (1 + r3) * MU_c3
    err_vec = np.array([err1, err2])
    
    return err_vec

We can now run a multivariate root finder to try and find the solution to this problem. The function [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html#scipy.optimize.root) contains all of the multivariate root finders in `SciPy`. Within `scipy.optimize.root()`, you can choose among 10 different methods of multivariate root finders.

In [ ]:
# Declare parameter values and exogenous variable values
beta = 0.96 ** (80 / 3)
sigma = 2.1
r2 = 0.6
r3 = 0.7
w1 = 0.5
w2 = 0.5
w3 = 0.5

# Make initial guess for solution of savings values. Note that these
# two guesses must be feasible and not violate c_t > 0 for all t
b2_init = 0.05
b3_init = 0.10
b_init = np.array([b2_init, b3_init])
b_args = (r2, r3, w1, w2, w3, beta, sigma)
b_result = opt.root(get_EulErrs, b_init, args=(b_args))
print(b_result)
print('Roots: ', b_result.x)

The output from the multivariate minimizer above is stored as a Python dictionary named `b_result`. The elements of the dictionary include:

* `fun`: the value of the error functions at the solution
* `message`: a message about whether the solution converged
* `nfev`: number of function evaluations until convergence
* `success`: boolean that you want to equal `True`
* `x`: the vector of roots of the function

## 3. Minimizers
[TODO]

### 3.1. Unconstrained minimizers
[TODO]

### 3.2. Constrained minimizers
[TODO]

## 4. Final Notes
The little advertised truth is that no root finder is guaranteed to find the right roots or any roots, and no minimizer is guaranteed to find the global minimum for any set of characterizing equations. More accurately, it is a rare and special case to find characterizing functions for which standard root finders and minimizers solve robustly. Press, et al (2007, pp. 442-443) highlight that with nonlinear multidimensional root finding problems,

> "...you can never be sure that the root is there at all until you have found it.... It cannot be overemphasized, however, how crucially success depends on having a good first guess for the solution,...."

Press, et al (2007, p. 473) also state,

> "We make an extreme, but wholly defensible statement: There are no good, general methods for solving systems of more than one nonlinear equation. Furthermore, it is not hard to see why (very likely) there never will be any good, general methods."

Because initial values are so important to root finders and minimization problems, Judd (1998, p. 172) suggests running a minimization problem with a loose stopping rule on the vector of squared errors. Then one can use that solution as the initial guess for the root finder.

Because root finding and minimization is so difficult, we take great care to report the errors and optimization output in our code. One must show not only that the minimizer or root finder stopped iterating, but also that the errors in the characterizing equations are arbitrarily small. One can also perform robustness tests using varying initial values to provide evidence that the solution is unique in the neighborhood of the solution.

## References
* Judd, Kenneth L., *Numerical Methods in Economics*, MIT Press (1998).
* Press, William H., Saul A. Teukolsky, William T. Vetterling, and Brian P. Flannery, *Numerical Recipes: The Art of Scientific Computing*, 3rd edition, Cambridge University Press (2007).